Uploading Phase:

In [ ]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy
!pip install scikit-learn

Calling the Neccessary Libraries:

In [ ]:
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from skimage import exposure
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import os

Finding Height and Width of an Image:

In [ ]:
sample_image_path = '/content/drive/MyDrive/FACE IMAGES/1.1.pgm'

# Load the image in grayscale
img = cv2.imread('/content/drive/MyDrive/FACE IMAGES/1.1.pgm', cv2.IMREAD_GRAYSCALE)

# Print its shape
print("Image shape:", img.shape)

height = img.shape[0]
print("Height:", height)

Image shape: (80, 64)
Height: 80


Training Part:

In [ ]:
import os
import cv2
import numpy as np
from skimage import exposure
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import normalize

# ----------------------------
# CONFIGURATION
# ----------------------------
DATASET_PATH = "/content/drive/MyDrive/FACE IMAGES"
IMAGE_EXTENSION = ".pgm"
NUM_SUBJECTS = 250
TRAIN_PER_SUBJECT = 2
target_size = (60, 80)

# LBP Parameters
P = 16
R = 2
PATCH_SIZE = 5
GRID_ROWS = 4
GRID_COLS = 4

# ----------------------------
# FUNCTION TO BUILD RIU2 MAPPING TABLE
# ----------------------------
def get_riu2_mapping(P):
    table = np.zeros(2**P, dtype=np.uint8)
    for i in range(2**P):
        binary = [(i >> j) & 1 for j in range(P)]
        binary.append(binary[0])  # wrap around
        transitions = sum(binary[j] != binary[j+1] for j in range(P))
        if transitions <= 2:
            table[i] = sum(binary[:P])  # count 1s
        else:
            table[i] = P + 1            # non-uniform pattern label
    return table

# ----------------------------
# CREATE RIU2 MAPPING
# ----------------------------
riu2_map = get_riu2_mapping(P)

# ----------------------------
# MAIN TRAINING LOOP (with Grid-Based LBP)
# ----------------------------
train_lbp_features = []

for subject_id in range(1, NUM_SUBJECTS + 1):
    for sample_id in range(1, TRAIN_PER_SUBJECT + 1):
        filename = f"{subject_id}.{sample_id}{IMAGE_EXTENSION}"
        img_path = os.path.join(DATASET_PATH, filename)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Could not load image: {filename}")
            continue

        resized = cv2.resize(img, target_size)
        resized_eq = exposure.equalize_hist(resized).astype(np.float64)
        resized_norm = (resized_eq - np.mean(resized_eq)) / np.std(resized_eq)

        feature_vector = []

        # Divide image into grid cells
        cell_h = target_size[1] // GRID_ROWS
        cell_w = target_size[0] // GRID_COLS

        for row in range(GRID_ROWS):
            for col in range(GRID_COLS):
                y_start = row * cell_h
                x_start = col * cell_w
                region = resized_norm[y_start:y_start+cell_h, x_start:x_start+cell_w]

                # Slide over region
                for y in range(0, region.shape[0] - PATCH_SIZE + 1, 1):
                    for x in range(0, region.shape[1] - PATCH_SIZE + 1, 1):
                        block = region[y:y+PATCH_SIZE, x:x+PATCH_SIZE]

                        # Step 1: Apply RoR
                        lbp_block_ror = local_binary_pattern(block, P, R, method='ror').astype(np.uint16)

                        # Step 2: Apply Uniform Mapping
                        lbp_block_uniform = riu2_map[lbp_block_ror]

                        # Step 3: Compute histogram
                        hist, _ = np.histogram(
                            lbp_block_uniform.ravel(),
                            bins=np.arange(0, P + 3),
                            density=True
                        )

                        feature_vector.extend(hist)

        train_lbp_features.append(feature_vector)
        print(f"✅ Processed {filename} | Feature length: {len(feature_vector)}")

# ----------------------------
# CONVERT TO NUMPY ARRAY + NORMALIZE
# ----------------------------
train_lbp_features = np.array(train_lbp_features, dtype=np.float32)
train_lbp_features = normalize(train_lbp_features, norm='l2')

# ----------------------------
# FINAL OUTPUT
# ----------------------------
print("\n✅ All training features extracted using Grid-based RoR + uniform mapping!")
print("🔢 Feature matrix shape:", train_lbp_features.shape)


/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


✅ Processed 1.1.pgm | Feature length: 50688
✅ Processed 1.2.pgm | Feature length: 50688
✅ Processed 2.1.pgm | Feature length: 50688
✅ Processed 2.2.pgm | Feature length: 50688
✅ Processed 3.1.pgm | Feature length: 50688
✅ Processed 3.2.pgm | Feature length: 50688
✅ Processed 4.1.pgm | Feature length: 50688
✅ Processed 4.2.pgm | Feature length: 50688
✅ Processed 5.1.pgm | Feature length: 50688
✅ Processed 5.2.pgm | Feature length: 50688
✅ Processed 6.1.pgm | Feature length: 50688
✅ Processed 6.2.pgm | Feature length: 50688
✅ Processed 7.1.pgm | Feature length: 50688
✅ Processed 7.2.pgm | Feature length: 50688
✅ Processed 8.1.pgm | Feature length: 50688
✅ Processed 8.2.pgm | Feature length: 50688
✅ Processed 9.1.pgm | Feature length: 50688
✅ Processed 9.2.pgm | Feature length: 50688
✅ Processed 10.1.pgm | Feature length: 50688
✅ Processed 10.2.pgm | Feature length: 50688
✅ Processed 11.1.pgm | Feature length: 50688
✅ Processed 11.2.pgm | Feature length: 50688
✅ Processed 12.1.pgm | Featu

Testing Part:

In [ ]:
TEST_PER_SUBJECT = 2
test_lbp_features = []
test_labels = []

GRID_ROWS = 4
GRID_COLS = 4

for subject_id in range(1, NUM_SUBJECTS + 1):
    for sample_id in range(TRAIN_PER_SUBJECT + 1, TRAIN_PER_SUBJECT + TEST_PER_SUBJECT + 1):

        filename = f"{subject_id}.{sample_id}{IMAGE_EXTENSION}"
        img_path = os.path.join(DATASET_PATH, filename)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Could not load test image: {filename}")
            continue

        resized = cv2.resize(img, target_size)
        resized_eq = exposure.equalize_hist(resized).astype(np.float64)
        resized_norm = (resized_eq - np.mean(resized_eq)) / np.std(resized_eq)

        feature_vector = []

        # Divide image into grid cells
        cell_h = target_size[1] // GRID_ROWS
        cell_w = target_size[0] // GRID_COLS

        for row in range(GRID_ROWS):
            for col in range(GRID_COLS):
                y_start = row * cell_h
                x_start = col * cell_w
                region = resized_norm[y_start:y_start+cell_h, x_start:x_start+cell_w]

                for y in range(0, region.shape[0] - PATCH_SIZE + 1, 1):
                    for x in range(0, region.shape[1] - PATCH_SIZE + 1, 1):
                        block = region[y:y+PATCH_SIZE, x:x+PATCH_SIZE]

                        # Step 1: Apply RoR
                        lbp_block_ror = local_binary_pattern(block, P, R, method='ror').astype(np.uint16)

                        # Step 2: Apply Uniform Mapping
                        lbp_block_uniform = riu2_map[lbp_block_ror]

                        # Step 3: Compute histogram
                        hist, _ = np.histogram(
                            lbp_block_uniform.ravel(),
                            bins=np.arange(0, P + 3),
                            density=True
                        )

                        feature_vector.extend(hist)

        test_lbp_features.append(feature_vector)
        test_labels.append(subject_id)
        print(f"✅ Processed test image: {filename}")

# Finalize test set
test_lbp_features = np.array(test_lbp_features, dtype=np.float32)
test_lbp_features = normalize(test_lbp_features, norm='l2')
test_labels = np.array(test_labels)

print("\n✅ All test features extracted (grid-based RoR + uniform + histogram)!")
print("🧪 Test feature matrix shape:", test_lbp_features.shape)
print("🎯 Test labels shape:", test_labels.shape)


/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


✅ Processed test image: 1.3.pgm
✅ Processed test image: 1.4.pgm
✅ Processed test image: 2.3.pgm
✅ Processed test image: 2.4.pgm
✅ Processed test image: 3.3.pgm
✅ Processed test image: 3.4.pgm
✅ Processed test image: 4.3.pgm
✅ Processed test image: 4.4.pgm
✅ Processed test image: 5.3.pgm
✅ Processed test image: 5.4.pgm
✅ Processed test image: 6.3.pgm
✅ Processed test image: 6.4.pgm
✅ Processed test image: 7.3.pgm
✅ Processed test image: 7.4.pgm
✅ Processed test image: 8.3.pgm
✅ Processed test image: 8.4.pgm
✅ Processed test image: 9.3.pgm
✅ Processed test image: 9.4.pgm
✅ Processed test image: 10.3.pgm
✅ Processed test image: 10.4.pgm
✅ Processed test image: 11.3.pgm
✅ Processed test image: 11.4.pgm
✅ Processed test image: 12.3.pgm
✅ Processed test image: 12.4.pgm
✅ Processed test image: 13.3.pgm
✅ Processed test image: 13.4.pgm
✅ Processed test image: 14.3.pgm
✅ Processed test image: 14.4.pgm
✅ Processed test image: 15.3.pgm
✅ Processed test image: 15.4.pgm
✅ Processed test image: 16.3

Recognition and Accuracy:

In [ ]:
# -------------------------
# RECOGNITION + ACCURACY (Manhattan Distance)
# -------------------------
correct_matches = 0
total_tests = len(test_labels)

for idx, test_feat in enumerate(test_lbp_features):
    distances = np.sum(np.abs(train_lbp_features - test_feat), axis=1)  # Manhattan distance
    best_match_idx = np.argmin(distances)

    predicted_label = (best_match_idx // TRAIN_PER_SUBJECT) + 1
    true_label = test_labels[idx]

    print(f"Test #{idx + 1} | True: {true_label} | Predicted: {predicted_label}")

    if predicted_label == true_label:
        correct_matches += 1

accuracy = (correct_matches / total_tests) * 100
print(f"\n✅ Recognition Accuracy (Manhattan): {accuracy:.2f}% ({correct_matches}/{total_tests})")


Test #1 | True: 1 | Predicted: 1
Test #2 | True: 1 | Predicted: 1
Test #3 | True: 2 | Predicted: 2
Test #4 | True: 2 | Predicted: 2
Test #5 | True: 3 | Predicted: 3
Test #6 | True: 3 | Predicted: 3
Test #7 | True: 4 | Predicted: 4
Test #8 | True: 4 | Predicted: 4
Test #9 | True: 5 | Predicted: 5
Test #10 | True: 5 | Predicted: 5
Test #11 | True: 6 | Predicted: 6
Test #12 | True: 6 | Predicted: 6
Test #13 | True: 7 | Predicted: 7
Test #14 | True: 7 | Predicted: 7
Test #15 | True: 8 | Predicted: 8
Test #16 | True: 8 | Predicted: 8
Test #17 | True: 9 | Predicted: 9
Test #18 | True: 9 | Predicted: 9
Test #19 | True: 10 | Predicted: 10
Test #20 | True: 10 | Predicted: 10
Test #21 | True: 11 | Predicted: 230
Test #22 | True: 11 | Predicted: 7
Test #23 | True: 12 | Predicted: 12
Test #24 | True: 12 | Predicted: 12
Test #25 | True: 13 | Predicted: 13
Test #26 | True: 13 | Predicted: 13
Test #27 | True: 14 | Predicted: 14
Test #28 | True: 14 | Predicted: 14
Test #29 | True: 15 | Predicted: 15
T

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
